<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/5%EC%9D%BC%EC%83%81%EC%8A%B9%EC%A2%85%EB%AA%A9%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests konlpy
!pip install Article
!pip install pykrx
!pip install pandas_market_calendars
!pip install feedparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 30.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement Article (from versions: none)
ERROR: No matching distribution found for Article
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.8 MB/s eta 0:00:00
  Created wheel for s

# 최근 5거래일 동안 20% 이상 상승한 종목의 1) 상승율, 시가총액 2) 뉴스 3) 검색량 증가율

In [2]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import requests
import re
import json
import pandas as pd
import pytz
import calendar
from IPython.display import Markdown, display
import feedparser

# 한국 시간(Asia/Seoul) 설정
seoul_tz = pytz.timezone("Asia/Seoul")

# 1) 최근 5거래일 중 20% 이상 상승한 종목의 상승률과 시가총액을 보여준다.
def get_krx_trading_range():
    krx = mcal.get_calendar('XKRX')
    today = datetime.now(seoul_tz).date()
    start_date = today - timedelta(days=30)
    trading_days = krx.valid_days(start_date, today)
    # 최근 5거래일 중 첫 번째(5일 전)와 마지막(최신 거래일)을 구함
    start_trading_day = trading_days[-5]
    last_trading_day = trading_days[-1]
    return start_trading_day.date(), last_trading_day.date()

start_date, end_date = get_krx_trading_range()
print(f"시작일: {start_date}, 종료일: {end_date}")

# 주가 변동 데이터 가져오기
df = stock.get_market_price_change(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"), "ALL")

# 등락률이 20% 이상인 종목 필터링 및 내림차순 정렬
rising_stocks = df[df['등락률'] >= 20].sort_values('등락률', ascending=False)

# 시가총액 정보 가져오기
market_cap = stock.get_market_cap(end_date.strftime("%Y%m%d"))

# 결과 출력 및 종목명 리스트 생성 (종목명(티커) 형식)
print("최근 5일간 20% 이상 상승한 종목:")
stock_names = []
name_to_ticker = {}  # 종목명과 티커 매핑
for ticker in rising_stocks.index:
    stock_name = stock.get_market_ticker_name(ticker)
    name_to_ticker[stock_name] = ticker
    change_rate = rising_stocks.loc[ticker, '등락률']
    cap = market_cap.loc[ticker, '시가총액']
    cap_billion = cap / 100_000_000  # 억원 단위로 변환
    print(f"{stock_name}({ticker}) (등락률: {change_rate:.2f}%, 시가총액: {cap_billion:.2f}억원)")
    stock_names.append(stock_name)

# 네이버 API 정보 (발급받은 값 입력)
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 날짜 설정 (트렌드 데이터: 최근 30일)
END_DATE = datetime.now(seoul_tz)
START_DATE = END_DATE - timedelta(days=30)

def get_trend_data(keywords):
    url = "https://openapi.naver.com/v1/datalab/search"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET,
        "Content-Type": "application/json"
    }
    # 최대 5개씩 나누어 API 호출
    chunk_size = 5
    keyword_chunks = [keywords[i:i + chunk_size] for i in range(0, len(keywords), chunk_size)]
    all_results = []
    for chunk in keyword_chunks:
        body = {
            "startDate": START_DATE.strftime("%Y-%m-%d"),
            "endDate": END_DATE.strftime("%Y-%m-%d"),
            "timeUnit": "date",
            "keywordGroups": [{"groupName": keyword, "keywords": [keyword]} for keyword in chunk]
        }
        response = requests.post(url, headers=headers, data=json.dumps(body))
        data = json.loads(response.text)
        if 'results' in data:
            all_results.extend(data["results"])
        else:
            print("Error: 'results' key not found in API response for chunk:", chunk)
            print("API Response:", response.text)
    return all_results

def analyze_trends(trend_data):
    df_list = []
    for item in trend_data:
        df = pd.DataFrame(item["data"])
        df["keyword"] = item["title"]
        df_list.append(df)
    if not df_list:
        print("Error: No data to analyze. Check the API response and parameters.")
        return pd.DataFrame()
    df_combined = pd.concat(df_list)
    df_pivot = df_combined.pivot(index="period", columns="keyword", values="ratio")
    # 최근 7일과 이전 7일의 평균 검색량 비교
    recent_7days = df_pivot.iloc[-7:].mean()
    previous_7days = df_pivot.iloc[-14:-7].mean()
    growth_rate = (recent_7days - previous_7days) / previous_7days * 100
    return growth_rate.sort_values(ascending=False)

# 트렌드 데이터 가져오기 및 분석
trend_data = get_trend_data(stock_names)
growth_rates = analyze_trends(trend_data)

print("\n검색량 증가율 (%):")
for keyword, rate in growth_rates.items():
    ticker = name_to_ticker.get(keyword, "N/A")
    print(f"{keyword}({ticker}): {rate:.2f}%")

# 상승종목의 뉴스 가져오기 (최근 7일)
news_end_date = datetime.now(seoul_tz)
news_start_date = news_end_date - timedelta(days=7)

headers_news = {
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}

def clean_text(text):
    text = text.replace("&amp;", "&")
    text = text.replace("quot;", '')
    text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text

for query in stock_names:
    ticker = name_to_ticker.get(query, "N/A")
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&sort=date&display=10"
    response = requests.get(url, headers=headers_news)
    if response.status_code == 200:
        news_data = response.json()
        print(f"\n최근 7일간 '{query}({ticker})' 관련 뉴스:")
        count = 0
        for item in news_data["items"]:
            # pubDate를 %z 포맷을 사용해 파싱 후 서울 시간으로 변환
            pub_date = datetime.strptime(item["pubDate"], "%a, %d %b %Y %H:%M:%S %z")
            pub_date = pub_date.astimezone(seoul_tz)
            if news_start_date <= pub_date <= news_end_date:
                count += 1
                title = clean_text(item["title"])
                link = item["link"]
                print(f"{count}. {pub_date.strftime('%Y-%m-%d')} [{title}]({link})")
        if count == 0:
            print("최근 7일간 관련 뉴스가 없습니다.")
    else:
        print("네이버 API 호출 실패:", response.status_code)

for query in stock_names:
    ticker = name_to_ticker.get(query, "N/A")
    # 구글 뉴스 RSS 피드 URL (검색어: 한국무브넥스)
    rss_url = f"https://news.google.com/rss/search?q={query}&{ticker}&hl=ko&gl=KR&ceid=KR:ko"

    # RSS 피드 파싱
    feed = feedparser.parse(rss_url)

    # 현재 시간과 7일 전 날짜 계산 (한국 시간 기준)
    now = datetime.now(seoul_tz)
    cutoff = now - timedelta(days=7)

    news_items = []

    # RSS 피드의 각 항목을 확인하여 지난 7일 이내의 뉴스만 선택 (최대 10개)
    for entry in feed.entries:
        # published_parsed는 time.struct_time 형식이므로 calendar.timegm을 사용해 timestamp로 변환
        ts = calendar.timegm(entry.published_parsed)
        published_dt = datetime.fromtimestamp(ts, seoul_tz)
        if published_dt >= cutoff:
            news_items.append((entry.title, entry.link))
        if len(news_items) >= 10:
            break

    # # 결과 출력 (헤드라인과 링크)
    # print(f"\n최근 7일간 '{query}({ticker})' 관련 뉴스:")
    # for idx, (title, link) in enumerate(news_items, start=1):
    #     print(f"{idx}. {title}\n   링크: {link}")
    # 결과를 Markdown 형식으로 생성하여 출력 (제목 클릭 시 링크로 연결, 링크는 노출되지 않음)
    markdown_text = f"\n최근 7일간 '{query}({ticker})' 관련 뉴스: (최대 10개):\n\n"
    for idx, (title, link) in enumerate(news_items, start=1):
        markdown_text += f"{idx}. [{title}]({link})\n\n"

    display(Markdown(markdown_text))


/usr/local/lib/python3.11/dist-packages/pandas_market_calendars/market_calendar.py:179: UserWarning: ['break_start', 'break_end'] are discontinued, the dictionary `.discontinued_market_times` has the dates on which these were discontinued. The times as of those dates are incorrect, use .remove_time(market_time) to ignore a market_time.
  warnings.warn(


시작일: 2025-01-24, 종료일: 2025-02-05


/usr/local/lib/python3.11/dist-packages/pykrx/website/krx/market/wrap.py:188: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', 0)


최근 5일간 20% 이상 상승한 종목:
에스에이티이엔지(351320) (등락률: 92.12%, 시가총액: 0.00억원)
아스테라시스(450950) (등락률: 71.52%, 시가총액: 0.00억원)
한국무브넥스(010100) (등락률: 65.24%, 시가총액: 0.00억원)
엔솔바이오사이언스(140610) (등락률: 62.30%, 시가총액: 0.00억원)
하이젠알앤엠(160190) (등락률: 59.70%, 시가총액: 0.00억원)
플리토(300080) (등락률: 57.50%, 시가총액: 0.00억원)
GRT(900290) (등락률: 50.91%, 시가총액: 0.00억원)
바이브컴퍼니(301300) (등락률: 49.75%, 시가총액: 0.00억원)
에이원알폼(234070) (등락률: 49.70%, 시가총액: 0.00억원)
디젠스(113810) (등락률: 42.00%, 시가총액: 0.00억원)
KS인더스트리(101000) (등락률: 39.88%, 시가총액: 0.00억원)
유니온(000910) (등락률: 37.87%, 시가총액: 0.00억원)
레인보우로보틱스(277810) (등락률: 37.10%, 시가총액: 0.00억원)
라온테크(232680) (등락률: 36.75%, 시가총액: 0.00억원)
카페24(042000) (등락률: 36.70%, 시가총액: 0.00억원)
진양화학(051630) (등락률: 36.04%, 시가총액: 0.00억원)
루트락(253610) (등락률: 35.73%, 시가총액: 0.00억원)
온코크로스(382150) (등락률: 34.72%, 시가총액: 0.00억원)
대한광통신(010170) (등락률: 34.00%, 시가총액: 0.00억원)
피델릭스(032580) (등락률: 32.00%, 시가총액: 0.00억원)
인스웨이브시스템즈(450520) (등락률: 31.28%, 시가총액: 0.00억원)
엠케이전자(033160) (등락률: 30.70%, 시가총액: 0.00억원)
코난테크놀로지(402030) (등락률: 29.90%, 시가총액: 0.00억원)
디와이디


최근 7일간 '에스에이티이엔지(351320)' 관련 뉴스: (최대 10개):

1. [[급등락주 짚어보기] 에스에이티이엔지, 최대주주 변경 기대에 이틀 연속 上 - 이투데이](https://news.google.com/rss/articles/CBMiVEFVX3lxTE91RGoxUndLdWJpNmhQaEZrNzJMZjBZQnFWODBxYzQ5VVd5LUVfejZGcm51SlZhaUZfZUtLUmJUOF9sSGtuZl96ZXl4YXBqZG9aVi1xQw?oc=5)

2. [에스에이티이엔지, 상한가 직행 배경은? - 영남일보](https://news.google.com/rss/articles/CBMia0FVX3lxTE5jS0M5Z1RoMWQ1S0hCbzZwbXRvcEx4bTYzd2NIYmRwQXV1QzNvaUFsSElyYlFjSUV2OWh4c0h3YUE3RXdEWDdFOVpuTGFBSlNaQnIybHJYMTR2enRpY1NNc1VFa2VRV1hPZmlj?oc=5)

3. [[급등주] 에스에이티이엔지, +30.00% 상한가 기록 - 전국매일신문](https://news.google.com/rss/articles/CBMibkFVX3lxTE8tNGhLQWN6TXJSa2FhQXdBWEwtOVM5NEcwdVdWRFZmMzNHS1BESlJ6cDNPRDhlZnppWmQzcnFNYllXSGQ1b2Y4RzdhWXZSWVpwQmd4Sjd0YWJsUmpSeTlXRVdaWVdWUjRrdzdrTlVB?oc=5)

4. [에스에이티이엔지·바이오로그디바이스·파인텍 등 5종목 상한가, 아남전자·풀무원·강원에너지·효성중공업 등 큰 폭 상승 마감 - 영남일보](https://news.google.com/rss/articles/CBMiY0FVX3lxTE5ENUd6eUhlX3VYVjlYQmx5R2drQXVQajBLeVNtaExCdjdFOTQtSVhra01JRXpiZnRQOFhFaW1JRG8wSXpnR0V0YlBZNC1CLUpiU2NPdUtyRkNMdXgtdDg2Qk1tYw?oc=5)

5. [[특징주] 에스에이티이엔지, 최대주주 변경 소식에 '상한가' - 서울와이어](https://news.google.com/rss/articles/CBMibEFVX3lxTE5tT1BBOGxldElRb3B5Qi1UdnU2aWprS2RuZzRKZy1NY2FsOHZVSEczeGhXUkxiaXhscHFxVFBmLXZnNk5GTEVpY1NMQy1RVXpQYjR2dlVDV2VycUtnSnh1czlXYVd4M3Jtc0JkLQ?oc=5)

6. [에스에이티이엔지, 자금조달·최대주주 변경 기대감에 2연속 上 - 동양뉴스](https://news.google.com/rss/articles/CBMiYEFVX3lxTE5JMFVhOFdPdll6OW4xVXh2OTdUaGdlVzJmQWRSTkQtczgtOHJiRkZ1Zng4UDh3Ny0tRHZYUS05NkduUkVVa29pWVNmbC0xWVN2MU4wcElfaE0xY0o0cEswdQ?oc=5)

7. [에스에이티이엔지, 상한가 진입.. +30.00% ↑ - 서울경제신문](https://news.google.com/rss/articles/CBMiVkFVX3lxTE1tWjEtaDQ2dzEwQXRuRHVac294bHFjX0gydlNrQmJfTlFVOVhvNnFfeXk0ZmlBWFM3aFhzeGJNN0FqYUNUWUNRQVZQNWZaaVd1Nmhwanln?oc=5)

8. [[특징주] 에스에이티이엔지, 최대주주 변경 소식에 상한가 - 글로벌이코노믹](https://news.google.com/rss/articles/CBMiiAFBVV95cUxNdUpoMjdkMzVRdi1WQlBpdWZrcFM1RTNLbGIzTHFwVE5kTzF6OUgwTVNrT2Nwa1JNRkpZYmJxNkJJUGRDbmV0bGhwcTY4c0VlWV94YmZrcElacm8ycGhjejBTQ2xOV2lNZFlGNmtfS3U5QU02TGtDbnV6T1pzRGRzYjZMOFg4bUw5?oc=5)

9. [[특징주] 에스에이티이엔지, 주식양수도 결정 소식에 `상한가` - 에너지경제](https://news.google.com/rss/articles/CBMiW0FVX3lxTE1pTVhyeFZud1VrU2RnaVU0TDFab2xmMFNLa1pSZDI3a20wQ3N6ZDVVRDFVLTNRdEx2aUJCR0FUc1p2allmSm9SNGdEbnJmZTBXMWRWSG5heWNicWs?oc=5)

10. [에스에이티이엔지, 자금조달·최대주주 변경 기대감에 2연속 上[핫스탁] - 뉴시스](https://news.google.com/rss/articles/CBMiYEFVX3lxTFBHMjU4QlVTalFRemVIcFdSSHlGMDZjZFhpWHFyNmljZHBOellkNGlHMk5nQlRLbkF3X3VUUG1TMUxrMmVjblNqaWxkWDE2RWxIT1Z3UkI4dnZoNUxibFZaV9IBeEFVX3lxTE96V3ZoQlJGckxlV2hhdXVuZ1FhbENXelRjc001VURYNzVpUTk2OFJWTDNmQTE2VzZKNlVValdyMDlDQkVRQU8yMGphTDl4QlMtNlYyWEZmaE1Ga194ZndYelpiLTc4bTFLUlh3UzFwY1BWc1doY1ZPdA?oc=5)



...


최근 7일간 '질경이(233990)' 관련 뉴스: (최대 10개):

1. [질경이, 젖은 이끼향 여성청결제 '그린 듀오' 출시 - 보건신문](https://news.google.com/rss/articles/CBMiY0FVX3lxTE5QalE5UVNIcHgtallEcGF0Q0xFdzN6WFB6d1RVUnRoTDY3X1VTWk1uVXV4ZUFpOU5GMlNNNGhrZEU5WlRjSi1sQVRTaEljd19FTlNXQ2g2S1A3SFc4eDJpdFRndw?oc=5)

2. [질경이, 여성청결제 그린 워시·그린 미스트 출시 - SR타임스](https://news.google.com/rss/articles/CBMiZ0FVX3lxTFBxa204WnBmNmg4VVRLb0VlOG1kQXdDOHc5bWJaQWxhUklrTDlOa2xjLXlWLTduZjU3d2ZSTUxMbzhQZGZ0SERJQWduRVFHT0lESmYtNm1QV181VExpSGZ1UmVBZUZYMHfSAWtBVV95cUxNTnNhNUlnQ2Z0QkdGUHdzRUc4VnJYTlNYcUpINTZ6LVJ3SEJTWkhUSVpKU0dwcXo1ZVRPazZMWGV3MWpoRnA1VjZqQ2NGM2NyT0VQSlJFLVJKUGxFcFNkTkJOOE54RGh0R1Jvbw?oc=5)

3. [질경이, 이끼 숲향 여성청결제 '그린 워시, 그린 미스트' 출시 - 싱글리스트](https://news.google.com/rss/articles/CBMiZkFVX3lxTE5xVHFUcmpzdEhMN3RVMkYtUkZxWExQcDQtZ3hfSElucXhNSkNRN3RycVNGQUVhTm43bGRBZDFfbHhxVGhqM3NiQmV2WE9rOHFKWk1qR3FvNmthLVNzSnhBa2VydjRwUQ?oc=5)




최근 7일간 '펨트론(168360)' 관련 뉴스: (최대 10개):

1. [특징주, 펨트론-반도체 장비 테마 상승세에 21.47% ↑ - 매일경제](https://news.google.com/rss/articles/CBMiUkFVX3lxTFBLR2Ezc2JQOTl1SXRaWFhyeEhYT2Nkck1CTGl1UmRqVDNIZGplV2JHeFhjQU5tVkUyVk1ad3liTTN0X0U4TVFWYmZyd1Q5RVdiX0E?oc=5)

2. [펨트론 주가 덩실덩실...외국인 대량매수 눈길 - 핀포인트뉴스](https://news.google.com/rss/articles/CBMic0FVX3lxTE1uVENvSGxjTGtIN0VvSFBoeEJUTEtuM2lEcmtSdGZfdGtoXzlaSG95SmliMXVGWG5IRFRQQ1hIN1VvUFpkd0F1dkEzSjRSY2pfZ3NQU0RsR3dWUWhRTlJEbzItUDBvamxmajFCTmdlNnpfWEHSAXdBVV95cUxNX3Q5bUgwNEM5cWlwT0JsLVk4aWpsZjNOWlJCekNSdzBPUFJqRWl1Y2lUaGZyaGZBR3ZTdDZEc2JhQ1ZYUFZoLXMxTmU2di1WUGtmQy1oVUpxeVpHNS0xcGN2VXRMUHBMay1XWU02NGVIZDJaaUtrTQ?oc=5)

3. [[강세 토픽] 2차전지 공정장비 테마, 펨트론 +21.33%, 파인텍 +10.18% - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxOZ2dwa2NYNHFnbFZiUEZGQ29wVmtYTUpMZ2c3OHdhWDBhS1ktRXgtY09TcWxXcXZQX1RaTHA4NU1GYUw4QmRxbWRCbGVieExQOTA1cVZTd2Fld2ZYcGlGaXVQaExSeC1xU28tSXZiRVdSdEh2UHlPcENURHIxUXg2Y1pKVdIBlwFBVV95cUxQeXUzendzbF9wUTBTRnJma3ZsZllvaGRnWnFDN2xzUFF3Tnl3RENyakpRXzRseHA2SFNwTm15VzhXcVExOXlhcnNYdVlhUlFBMjE5QW9EZzNqV2NiemYyU2dNRloxSFZOWTV4TUdfeS1oTGZBbVdFVmdpbjFnTjZUZXpUbEJaejFVQ1NGWmIyM2Y2b29NbmZ3?oc=5)

4. [펨트론, +11.82% VI 발동 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxOclRmb3Y5VXlnYXZUQWh0VXl4cHpkRmF2TFZaLVlYaC00RERsTmVkV0tqSUVmT1JQbmE4cE1UUS1VeHg4Z2ZhVU9SRWtqMWhobzh5T3lkWVd1bG5yX2c5TFpiY0FldGQzNlNIcWFac3RteGhzQ3Q1OFluWDlEeGNsNlVrUdIBlwFBVV95cUxOQlpBYnpDV1VGOTBVTUcwRU1DRzBBQ1VCQ3k3STZ2cjhGand1bDNlTWdZTGRHMUZYWjVDNElwWEpzTkZZZ2MxbUJRbjdScFpsOHQzelRraHNaQ2NidHROM3lIZ1liOEhGdkNYeXBBUWlKNHp4cjZRZFRadUxpMUJNb3ZfQlR2YXgtYTg2MHBkcU1lcnJUajI4?oc=5)

5. [특징주, 저스템-반도체 장비 테마 상승세에 5.76% ↑ - 매일경제](https://news.google.com/rss/articles/CBMiUkFVX3lxTE1CcTZ1cHlXZ2FybVlxeFdUVi1aVXVPNExIVE1td2hmdml3WFlWMkktMXJuSE1pazN4eU5LS3kyNVplUzdHTllaanV5WHJRS3BRX1E?oc=5)

6. [펨트론, 27% 급등…오로스테크놀로지·티에프이·저스템·기가비스 등 반도체 장비株 강세 - 영남일보](https://news.google.com/rss/articles/CBMia0FVX3lxTE15ZG5ZUEFWQkZTY0xBU3kzb3RlcXY2ZjBWVGdxYmJRTnVIWnBoZTVqWVdUOXJSS3oySmR5ZUgta1g1ejlmdlMxTU9SdzhSdUFwQjR0TXItcFByWFBpQTNfaHVaczhuc0dYQUE4?oc=5)

7. [[생생한 주식쇼 생쇼] 트럼프 관세 전략과 AI·반도체·바이오 주요 종목 리뷰 / 펨트론 (168360) - 매일경제TV](https://news.google.com/rss/articles/CBMiaEFVX3lxTE5qMkgxdzFtb0FGbDNyMVV1UUF4ektSZGlnUEo3TmJGcDlKQ0ZrX2JBWkJrNkNXbGdqTWxjYTJ1Vl9KUjJkdXBtSzVVTEpaclY0WWtFcG5vSmUybmgza1p4WDdQY3RrSFlC?oc=5)

8. [펨트론, 주가와 거래량 모두 강세... 최근 5거래일 최고 거래량 돌파 - 조세금융신문](https://news.google.com/rss/articles/CBMiYEFVX3lxTE1nNHZIRmRESTNXUURhSDhNNHpRNWxvRkdZZ3I5RldkN2dPVGl2ZkZXRm9yV2dmVnZtQzVCMENSQ19BOWx6b2FfYnFBTkc1eXg1ZzR5cTBYbElSU2lLUTZUSA?oc=5)

9. [특징주, 펨트론-반도체 장비 테마 상승세에 21.47% ↑ - MK빌리어드뉴스](https://news.google.com/rss/articles/CBMiU0FVX3lxTFBEQzg3NURYVEp3cFRiRTc2ckN4c25QUDdDNHo0Q3RLUG8yTFFXdnZ4WnAtOGVzN3J1WDlTcHFmeDFUOFpkcU9MOEZUMWJDN3R2REow?oc=5)

10. [특징주, 엑시콘-반도체 장비 테마 상승세에 5.57% ↑ - MK빌리어드뉴스](https://news.google.com/rss/articles/CBMiU0FVX3lxTE9LdXpMX19FbkI2b184S19wTm5KMGxMcTJGWlU1WjloNnZ6TmZtX0UybXFTVTN0emFBajhGUk5hVEVmcUlxUzctVV9CLTg2ZlN1SFU0?oc=5)




최근 7일간 '태양금속(004100)' 관련 뉴스: (최대 10개):

1. [태양금속우, +5.10% 상승폭 확대 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxPRzNhMW01cHBHX181S2w2VUF3R2JRQ1V6UzdvdlBzOWg4bHpXb3daTXpLYlNPVWRuclFFSWU4WEhoQS1QZGNiS0NyWjA1VmduRlRFYUNyR2NOeXlSRWNtN19HdHd4T1hqOFk2dzRXWmFic1F2UkEyMDREWWdLQWVhUWQ0NNIBlwFBVV95cUxQbENlTVZEd0JkSXpLcnlHeUdwM1JRcHlPeFZxUnY0R1dBWVlXUTVlRmhGc2tmUl9iMEpwSTMtNHAwazlNTzdYYWs4QXh4emFHMG9WSjJ5azhLWGhuRng0WE1xb2FVcW9tRF91NVJGWm1QVzNBRTZoUUMtWEE0aWRKTjRoR1Z3QjVOTnhtNldOaHB3d0gwa3BN?oc=5)




최근 7일간 '휴먼테크놀로지(175140)' 관련 뉴스: (최대 10개):




최근 7일간 '에스비비테크(389500)' 관련 뉴스: (최대 10개):

1. [[로봇사업 기대株] 에스비비테크, 로봇 감속기 국산화 선도 - 딜사이트](https://news.google.com/rss/articles/CBMiT0FVX3lxTFBBRUZycjRpM3dKaE9jQmpKYVhRX3JtUlVTMjVMTGFybE5JamE1dmN1TFFkVm5aU18tMVRoWEZDNFdVVFlwQzZxUUJ5Tm9leGM?oc=5)

2. [에스비비테크, +9.90% VI 발동 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxObjAxRUt4WkthVzhWaWRDdW1DUnNsYTVMbzZodWt0VkJaWkNfbzd5UGtVejlNRGlLb0FyNWtPZDZ0VFQ2LXFPQlJVRmZTY1JsQlZ3UktSODltMkxTVHBRQTFKZ01BYXQ5aWlFMDlkN04yaVF4Z0k3NlhVbmpfeFZnV2Rkd9IBlwFBVV95cUxNQjV2QjNyVkNZdUZGQ241c29qLTdWVmdDZVlqQTkwRkpmMDB2WG13VkFoMnp5Mk5vOXY2RjZpRXVSRlc4eFQyN2RzVm9CYXgwUE9teW5iZk5rVkg2OWM2WXpNWThTRGV0b253bGoxSmphNUlENjUzaExndW9hNjE0WXI0Y0RfVEkzT1N1dVU2NjFXR0RnLTVn?oc=5)

3. [[로봇사업 기대株] 에스비비테크, 로봇 감속기 국산화 선도 - 딜사이트 플러스](https://news.google.com/rss/articles/CBMiVEFVX3lxTE93NDM3cTRTMl9tamlDZUJqaklnaC1FYmVid1Z4SU5Ic2pKMEZINFdtb1BBYkcxZU9HVHR5TnYxN2ZpTVNlaXpIM3FUN3RLVm0wb0VIWg?oc=5)

4. [특징주, 에스비비테크-로봇(산업용/협동로봇 등) 테마 상승세에 7.1% ↑ - 매일경제](https://news.google.com/rss/articles/CBMiRkFVX3lxTE5wQ3JNeVhTSVMteGplZnVIMmNnYzdoWm5DVVcybmdJeVpDX05QS1FrVnNZR0xPcjU1bHFzY25IMmVYUzR1cUE?oc=5)

5. [[로봇사업 기대株] 에스비비테크, 로봇 감속기 국산화 선도 - 딜사이트](https://news.google.com/rss/articles/CBMiWEFVX3lxTE1tZDZnMDA0RDhBenpsZ0Z6bVNWQmNIbzB0dXFtT3l2bDdfUUp2YzI4cDNRbGNfdWVHaUNCMGJVVkI5YTBxSjUxSjNFZmM0Q3JWeWhFbkc1LXU?oc=5)



#5일간 20% 이상 상승한 종목의 60일간 주가데이터를 받아옴


In [ ]:
!pip install pykrx

In [ ]:
import pandas as pd
from pykrx import stock
from datetime import datetime

def get_stock_data_pivot(stock_list):
    final_df = pd.DataFrame()
    max_prices = {}
    min_prices = {}
    max_rates = {}
    min_rates = {}
    max_dates = {}
    min_dates = {}
    # company_names = [s['종목명'] for s in stock_list]  # 종목명 추출
    company_names = []

    for i, stock_info in enumerate(stock_list[:]):  # 최대 150개 항목 처리
        company_name = f"{stock_info['종목명']}_{i}"  # 고유 이름 생성
        company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        # print(i, company_name)  # 진행 상황 확인

        # 종목 코드 찾기
        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
            continue

        # 주가 데이터 가져오기
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        df = df[['종가']].rename(columns={'종가': company_name})


        # 최고가/최저가 계산
        max_prices[company_name] = df[company_name].max()
        min_prices[company_name] = df[company_name].min()
        max_rates[company_name] = round(((df[company_name].max()/df[company_name].iloc[0])-1)*100,2)
        min_rates[company_name] = round(((df[company_name].min()/df[company_name].iloc[0])-1)*100,2)
        max_dates[company_name] = df[company_name].idxmax().strftime('%Y-%m-%d')
        min_dates[company_name] = df[company_name].idxmin().strftime('%Y-%m-%d')
        # print(df[company_name][0])

        # 데이터 병합
        final_df = pd.merge(final_df, df, left_index=True, right_index=True, how='outer') if not final_df.empty else df
        # print(final_df)
    # 추가 행 생성 (수정)
    additional_data = {
    '종목별': ['최저주가', '최고주가', '최저하락률', '최고상승률', '최저주가일', '최고주가일'],
    **{name: [
        min_prices.get(name, 'N/A'),
        max_prices.get(name, 'N/A'),
        min_rates.get(name, 'N/A'),
        max_rates.get(name, 'N/A'),
        min_dates.get(name, 'N/A'),
        max_dates.get(name, 'N/A')
    ] for name in company_names}
}
    additional_df = pd.DataFrame(additional_data).set_index('종목별')

    return pd.concat([final_df, additional_df])

# # 입력 데이터
# stocks = [
#     {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
#     {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
#     {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
#     {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
#     {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
# ]



# 엑셀 파일 읽기
df = pd.read_excel("/종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
# print(df.head(230))
# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })


result_df = get_stock_data_pivot(stocks)
result_df.to_excel("stock_analysis_final.xlsx")
print(result_df)


0 위메이드_0
1 RFHIC_1
2 야스_2
3 두산_3
4 코오롱티슈진_4
5 브릿지바이오테라퓨틱스_5
6 코스모화학_6
7 롯데케미칼_7
8 제이앤티씨_8
9 대한유화_9
10 효성티앤씨_10
11 송원산업_11
12 코스모신소재_12
13 대주전자재료_13
14 엘앤에프_14
15 LS_15
16 한진칼_16
17 바이오다인_17
18 원준_18
19 HPSP_19
20 에이스테크_20
21 HD현대일렉트릭_21
22 영풍정밀_22
23 고려아연_23
24 유한양행_24
25 레인보우로보틱스_25
26 HD현대에너지솔루션_26
27 레이크머티리얼즈_27
28 티앤엘_28
29 솔브레인홀딩스_29
30 엘티씨_30
31 SK하이닉스_31
32 엔켐_32
33 인탑스_33
34 이엔에프테크놀로지_34
35 GST_35
36 월덱스_36
37 한화솔루션_37
38 SBI핀테크솔루션즈_38
39 디앤씨미디어_39
40 주성엔지니어링_40
41 제우스_41
42 이수페타시스_42
43 휴젤_43
44 제넥신_44
45 리노공업_45
46 크라우드웍스_46
47 ISC_47
48 SK오션플랜트_48
49 녹십자_49
50 뷰노_50
51 휴메딕스_51
52 SK바이오팜_52
53 씨에스윈드_53
54 파마리서치_54
55 헬릭스미스_55
56 이엠텍_56
57 태영건설_57
58 HD현대중공업_58
59 올릭스_59
60 F&F_60
61 성광벤드_61
62 GS건설_62
63 태광_63
64 HD현대건설기계_64
65 다산네트웍스_65
66 코오롱티슈진_66
67 두산로보틱스_67
68 효성중공업_68
69 브릿지바이오테라퓨틱스_69
70 케이씨텍_70
71 유니테크노_71
72 푸드나무_72
73 삼천당제약_73
74 넥슨게임즈_74
75 솔루스첨단소재_75
76 이랜텍_76
77 더존비즈온_77
78 메디톡스_78
79 코웨이_79
80 세진중공업_80
81 GS_81
82 DB하이텍_82
83 LIG넥스원_83
84 코스맥스_84
85 화신_85
86 서연

#엑셀자료 json변환

In [ ]:
import pandas as pd

# 엑셀 파일 읽기
df = pd.read_excel("/종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
print(df)
# DataFrame을 JSON 형식으로 변환
json_data = []
i = 0
for _, row in df.iterrows():
    json_data.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })
    i += 1

# JSON 형식으로 출력
print("[")
for i, item in enumerate(json_data):
    if i < len(json_data) - 1:
        print(f"    {str(item)},")
    else:
        print(f"    {str(item)}")
print("]")

          종목명        매수일        매도일     종목코드       수익률
0        위메이드 2024-10-30 2025-01-31  A112040  0.025641
1       RFHIC 2024-10-25 2025-01-22  A218410  0.029903
2          야스 2024-10-24 2025-01-21  A255440 -0.203985
3          두산 2024-10-18 2025-01-15  A000150  0.330097
4      코오롱티슈진 2024-10-17 2025-01-14  A950160  0.848559
...       ...        ...        ...      ...       ...
1809      제우스 2017-01-06 2017-04-05  A079370  0.087818
1810      HLB 2017-01-05 2017-04-04  A028300  0.003252
1811  뉴파워프라즈마 2017-01-02 2017-03-30  A144960 -0.004357
1812   대주전자재료 2017-01-02 2017-03-30  A078600  0.047674
1813    동아에스티 2017-01-02 2017-03-30  A170900 -0.105266

[1814 rows x 5 columns]
[
    {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
    {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
    {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
    {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
    {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},

#종목명 체크

In [10]:
import pandas as pd
from pykrx import stock
from datetime import datetime

def get_stock_data_pivot(stock_list):
    # final_df = pd.DataFrame()
    # max_prices = {}
    # min_prices = {}
    # max_rates = {}
    # min_rates = {}
    # max_dates = {}
    # min_dates = {}
    # # company_names = [s['종목명'] for s in stock_list]  # 종목명 추출
    # company_names = []

    for i, stock_info in enumerate(stock_list[:]):  # 최대 150개 항목 처리
        company_name = f"{stock_info['종목명']}_{i}"  # 고유 이름 생성
        # company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        # print(i, company_name)  # 진행 상황 확인

        # 종목 코드 찾기
        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {i} {stock_info['종목명']}")
            continue

    return


# 엑셀 파일 읽기
df = pd.read_excel("/content/5일20%_all.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
# print(df.head(230))
# DataFrame을 JSON 형식으로 변환
stocks = []
# Handle NaT values for '매수일' and '매도일'
df['매수일'] = df['매수일'].fillna(pd.NaT)
df['매도일'] = df['매도일'].fillna(pd.NaT)

# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d') if not pd.isna(row['매수일']) else None,
        "매도일": row['매도일'].strftime('%Y-%m-%d') if not pd.isna(row['매도일']) else None,
    })

# print(stocks)

get_stock_data_pivot(stocks)
# result_df.to_excel("stock_analysis_final.xlsx")
# print(result_df)


종목을 찾을 수 없습니다: 1 한화인더스트리얼솔루션즈
종목을 찾을 수 없습니다: 142 에스유홀딩스
종목을 찾을 수 없습니다: 179 비트나인
종목을 찾을 수 없습니다: 199 STX중공업
종목을 찾을 수 없습니다: 258 스카이문스테크놀로지
종목을 찾을 수 없습니다: 347 젬백스링크
종목을 찾을 수 없습니다: 364 윈텍
종목을 찾을 수 없습니다: 391 와이아이케이
종목을 찾을 수 없습니다: 433 룽투코리아
종목을 찾을 수 없습니다: 446 엔터파트너즈
종목을 찾을 수 없습니다: 459 에이프로젠 H&G
종목을 찾을 수 없습니다: 468 하인크코리아
종목을 찾을 수 없습니다: 473 이노시스
종목을 찾을 수 없습니다: 514 에스유홀딩스
종목을 찾을 수 없습니다: 519 리노스
종목을 찾을 수 없습니다: 551 젬백스링크
종목을 찾을 수 없습니다: 577 롯데정보통신
종목을 찾을 수 없습니다: 578 인포마크
종목을 찾을 수 없습니다: 611 비플라이소프트
종목을 찾을 수 없습니다: 632 트루윈
종목을 찾을 수 없습니다: 639 동일철강
종목을 찾을 수 없습니다: 687 에이치앤비디자인
종목을 찾을 수 없습니다: 706 스피어파워
종목을 찾을 수 없습니다: 716 대유에이피
종목을 찾을 수 없습니다: 726 스카이문스테크놀로지
종목을 찾을 수 없습니다: 748 디딤이앤에프
종목을 찾을 수 없습니다: 750 바이옵트로
종목을 찾을 수 없습니다: 771 LS전선아시아
종목을 찾을 수 없습니다: 860 텔레필드
종목을 찾을 수 없습니다: 894 SBW생명과학
종목을 찾을 수 없습니다: 917 오하임아이엔티
종목을 찾을 수 없습니다: 936 KT서브마린
종목을 찾을 수 없습니다: 962 에스유홀딩스
종목을 찾을 수 없습니다: 973 STX중공업
종목을 찾을 수 없습니다: 992 덕신하우징
종목을 찾을 수 없습니다: 1009 알비케이그룹
종목을 찾을 수 없습니다: 1017 브이티지엠피
종목을 찾을 수 없습니다: 1024 에이트원
종목을 찾을 수 없습니다: 10

TypeError: strptime() argument 1 must be str, not None

# 인덱스 리셋

In [ ]:
!pip install pykrx

In [ ]:
import pandas as pd
from pykrx import stock
from datetime import datetime

def get_stock_data_pivot(stock_list):
    final_df = pd.DataFrame()
    max_prices = {}
    min_prices = {}
    max_rates = {}
    min_rates = {}
    max_dates = {}
    min_dates = {}
    # company_names = [s['종목명'] for s in stock_list]  # 종목명 추출
    company_names = []

    for i, stock_info in enumerate(stock_list[:]):  # 최대 150개 항목 처리
        company_name = f"{stock_info['종목명']}_{i}"  # 고유 이름 생성
        company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        print(i, company_name)  # 진행 상황 확인

        # 종목 코드 찾기
        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
            continue

        # 주가 데이터 가져오기
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        df = df[['종가']].rename(columns={'종가': company_name})
        df.reset_index(drop=True, inplace=True)

        # 최고가/최저가 계산
        max_prices[company_name] = df[company_name].max()
        min_prices[company_name] = df[company_name].min()
        max_rates[company_name] = round(((df[company_name].max()/df[company_name].iloc[0])-1)*100,2)
        min_rates[company_name] = round(((df[company_name].min()/df[company_name].iloc[0])-1)*100,2)
        max_dates[company_name] = df[company_name].idxmax()
        min_dates[company_name] = df[company_name].idxmin()
        # print(df[company_name][0])

        # 데이터 병합
        final_df = pd.merge(final_df, df, left_index=True, right_index=True, how='outer') if not final_df.empty else df
        # print(final_df)
    # 추가 행 생성 (수정)
    additional_data = {
    '종목별': ['최저주가', '최고주가', '최저하락률', '최고상승률', '최저주가일', '최고주가일'],
    **{name: [
        min_prices.get(name, 'N/A'),
        max_prices.get(name, 'N/A'),
        min_rates.get(name, 'N/A'),
        max_rates.get(name, 'N/A'),
        min_dates.get(name, 'N/A'),
        max_dates.get(name, 'N/A')
    ] for name in company_names}
}
    additional_df = pd.DataFrame(additional_data).set_index('종목별')

    return pd.concat([final_df, additional_df])

# # 입력 데이터
# stocks = [
#     {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
#     {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
#     {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
#     {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
#     {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
# ]



# 엑셀 파일 읽기
df = pd.read_excel("종목리스트_5일_20%.xlsx", "Sheet2")  # 실제 엑셀 파일명으로 변경해주세요
# print(df.head(230))
# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d'),
        "매도일": row['매도일'].strftime('%Y-%m-%d')
    })


result_df = get_stock_data_pivot(stocks)
result_df.to_excel("stock_analysis_final.xlsx")
print(result_df)


0 위메이드_0
1 RFHIC_1
2 야스_2
3 두산_3
4 코오롱티슈진_4
5 브릿지바이오테라퓨틱스_5
6 코스모화학_6
7 롯데케미칼_7
8 제이앤티씨_8
9 대한유화_9
10 효성티앤씨_10
11 송원산업_11
12 코스모신소재_12
13 대주전자재료_13
14 엘앤에프_14
15 LS_15
16 한진칼_16
17 바이오다인_17
18 원준_18
19 HPSP_19
20 에이스테크_20
21 HD현대일렉트릭_21
22 영풍정밀_22
23 고려아연_23
24 유한양행_24
25 레인보우로보틱스_25
26 HD현대에너지솔루션_26
27 레이크머티리얼즈_27
28 티앤엘_28
29 솔브레인홀딩스_29
30 엘티씨_30
31 SK하이닉스_31
32 엔켐_32
33 인탑스_33
34 이엔에프테크놀로지_34
35 GST_35
36 월덱스_36
37 한화솔루션_37
38 SBI핀테크솔루션즈_38
39 디앤씨미디어_39
40 주성엔지니어링_40
41 제우스_41
42 이수페타시스_42
43 휴젤_43
44 제넥신_44
45 리노공업_45
46 크라우드웍스_46
47 ISC_47
48 SK오션플랜트_48
49 녹십자_49
50 뷰노_50
51 휴메딕스_51
52 SK바이오팜_52
53 씨에스윈드_53
54 파마리서치_54
55 헬릭스미스_55
56 이엠텍_56
57 태영건설_57
58 HD현대중공업_58
59 올릭스_59
60 F&F_60
61 성광벤드_61
62 GS건설_62
63 태광_63
64 HD현대건설기계_64
65 다산네트웍스_65
66 코오롱티슈진_66
67 두산로보틱스_67
68 효성중공업_68
69 브릿지바이오테라퓨틱스_69
70 케이씨텍_70
71 유니테크노_71
72 푸드나무_72
73 삼천당제약_73
74 넥슨게임즈_74
75 솔루스첨단소재_75
76 이랜텍_76
77 더존비즈온_77
78 메디톡스_78
79 코웨이_79
80 세진중공업_80
81 GS_81
82 DB하이텍_82
83 LIG넥스원_83
84 코스맥스_84
85 화신_85
86 서연

#종목별 최종, 최고, 최저만 기록

In [1]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 13.1 MB/s eta 0:00:00


In [8]:
import pandas as pd
from pykrx import stock
from datetime import datetime

time = datetime.now(seoul_tz)
print(time)

def get_stock_data_summary(stock_list):
    max_prices = {}
    min_prices = {}
    max_rates = {}
    min_rates = {}
    max_dates = {}
    min_dates = {}
    final_rates = {}
    start_prices = {}
    end_prices = {}
    company_names = []

    for i, stock_info in enumerate(stock_list[6020:]):
        company_name = f"{stock_info['종목명']}_{i}"
        company_names.append(company_name)
        start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
        end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

        print(i, company_name)

        # time1 = datetime.now(seoul_tz)
        # print(time1)

        tickers = stock.get_market_ticker_list(start_date, "ALL")
        ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)
        # print(ticker)
        # time2 = datetime.now(seoul_tz)
        # print(time2)

        if ticker is None:
            print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
            continue

        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        df.reset_index(drop=True, inplace=True)
        # print(df)
        # time3 = datetime.now(seoul_tz)
        # print(time3)

        max_prices[company_name] = df['종가'].max()
        min_prices[company_name] = df['종가'].min()
        max_rates[company_name] = round(((df['종가'].max()/df['종가'].iloc[0])-1)*100, 2)
        min_rates[company_name] = round(((df['종가'].min()/df['종가'].iloc[0])-1)*100, 2)
        max_dates[company_name] = df['종가'].idxmax()
        min_dates[company_name] = df['종가'].idxmin()
        final_rates[company_name] = round(((df['종가'].iloc[-1]/df['종가'].iloc[0])-1)*100, 2)
        start_prices[company_name] = df['종가'].iloc[0]
        end_prices[company_name] = df['종가'].iloc[-1]


    additional_data = {
        '종목별': ['시작일주가', '종료일주가', '최저주가', '최고주가', '최저하락률', '최고상승률', '최저주가일', '최고주가일', '최종수익률'],
        **{name: [
            start_prices.get(name, 'N/A'),
            end_prices.get(name, 'N/A'),
            min_prices.get(name, 'N/A'),
            max_prices.get(name, 'N/A'),
            min_rates.get(name, 'N/A'),
            max_rates.get(name, 'N/A'),
            min_dates.get(name, 'N/A'),
            max_dates.get(name, 'N/A'),
            final_rates.get(name, 'N/A'),
        ] for name in company_names}
    }
    print(additional_data)
    # time4 = datetime.now(seoul_tz)
    # print(time4)
    return pd.DataFrame(additional_data).set_index('종목별')

# 입력 데이터
# stocks = [
#     {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
#     {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
#     {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
#     {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
#     {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
# ]


# 엑셀 파일 읽기
df = pd.read_excel("/content/5일20%_all.xlsx", "Sheet2")
stocks = []
# Handle NaT values for '매수일' and '매도일'
df['매수일'] = df['매수일'].fillna(pd.NaT)
df['매도일'] = df['매도일'].fillna(pd.NaT)

# DataFrame을 JSON 형식으로 변환
stocks = []
for _, row in df.iterrows():
    stocks.append({
        "종목명": row['종목명'],
        "매수일": row['매수일'].strftime('%Y-%m-%d') if not pd.isna(row['매수일']) else None,
        "매도일": row['매도일'].strftime('%Y-%m-%d') if not pd.isna(row['매도일']) else None,
    })

result_df = get_stock_data_summary(stocks)
print(result_df)
time5 = datetime.now(seoul_tz)
print(time5)
result_df.to_excel("stock_summary_final.xlsx")


2025-02-05 15:48:32.950329+09:00
0 이엠넷_0
2025-02-05 15:48:35.234648+09:00
123570
2025-02-05 15:48:36.199628+09:00
      시가    고가    저가    종가      거래량        등락률
0   2765  2835  2680  2820   692603   1.256732
1   2800  2835  2695  2725   530986  -3.368794
2   2680  2760  2630  2710   469659  -0.550459
3   2735  3520  2715  3520  8270362  29.889299
4   3475  3475  3380  3450   742686  -1.988636
..   ...   ...   ...   ...      ...        ...
56  2920  3275  2890  2935  1233761   1.032702
57  2995  3090  2925  3010   519819   2.555366
58  3050  3185  3000  3015   607239   0.166113
59  3015  3100  2985  3100   320332   2.819237
60  3175  3680  3170  3370  7432075   8.709677

[61 rows x 6 columns]
2025-02-05 15:48:37.458815+09:00
1 마니커_1
2025-02-05 15:48:37.459649+09:00
027740
2025-02-05 15:48:38.262420+09:00
      시가    고가    저가    종가     거래량       등락률
0   4392  4417  4019  4171  888015 -1.789498
1   4108  4153  3886  3957  898231 -5.130664
2   3944  4030  3853  3909  655760 -1.213040
3   3

In [6]:
import pandas as pd
from pykrx import stock
from datetime import datetime
import multiprocessing as mp
from functools import lru_cache
import pytz

seoul_tz = pytz.timezone('Asia/Seoul')

@lru_cache(maxsize=128)
def cached_get_ticker_list(date):
    return stock.get_market_ticker_list(date, "ALL")

def process_stock(stock_info, idx):
    company_name = f"{stock_info['종목명']}_{idx}"
    start_date = datetime.strptime(stock_info['매수일'], "%Y-%m-%d").strftime("%Y%m%d")
    end_date = datetime.strptime(stock_info['매도일'], "%Y-%m-%d").strftime("%Y%m%d")

    print(f"Processing {idx}: {company_name}")

    tickers = cached_get_ticker_list(start_date)
    ticker = next((t for t in tickers if stock.get_market_ticker_name(t) == stock_info['종목명']), None)

    if ticker is None:
        print(f"종목을 찾을 수 없습니다: {stock_info['종목명']}")
        return None

    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df.reset_index(drop=True, inplace=True)

    if df.empty:
        print(f"데이터가 없습니다: {stock_info['종목명']}")
        return None

    start_price = df['종가'].iloc[0]
    end_price = df['종가'].iloc[-1]
    max_price = df['종가'].max()
    min_price = df['종가'].min()

    return {
        'company': company_name,
        'start_price': start_price,
        'end_price': end_price,
        'min_price': min_price,
        'max_price': max_price,
        'min_rate': round(((min_price / start_price) - 1) * 100, 2),
        'max_rate': round(((max_price / start_price) - 1) * 100, 2),
        'min_date': df['종가'].idxmin(),
        'max_date': df['종가'].idxmax(),
        'final_rate': round(((end_price / start_price) - 1) * 100, 2)
    }

def get_stock_data_summary(stock_list):
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.starmap(process_stock, [(s, i) for i, s in enumerate(stock_list)])

    results = [r for r in results if r is not None]

    if not results:
        return pd.DataFrame()

    df = pd.DataFrame(results)
    df.set_index('company', inplace=True)

    return df.T

if __name__ == "__main__":
    start_time = datetime.now(seoul_tz)
    print(f"시작 시간: {start_time}")

    # stocks = [
    #     {'종목명': '위메이드', '매수일': '2024-10-30', '매도일': '2025-01-31'},
    #     {'종목명': 'RFHIC', '매수일': '2024-10-25', '매도일': '2025-01-22'},
    #     {'종목명': '야스', '매수일': '2024-10-24', '매도일': '2025-01-21'},
    #     {'종목명': '두산', '매수일': '2024-10-18', '매도일': '2025-01-15'},
    #     {'종목명': '코오롱티슈진', '매수일': '2024-10-17', '매도일': '2025-01-14'},
    # ]

    # 엑셀 파일 읽기
    df = pd.read_excel("/content/5일20%_all.xlsx", "Sheet2")
    stocks = []
    # Handle NaT values for '매수일' and '매도일'
    df['매수일'] = df['매수일'].fillna(pd.NaT)
    df['매도일'] = df['매도일'].fillna(pd.NaT)

    # DataFrame을 JSON 형식으로 변환
    stocks = []
    for _, row in df.iterrows():
        stocks.append({
            "종목명": row['종목명'],
            "매수일": row['매수일'].strftime('%Y-%m-%d') if not pd.isna(row['매수일']) else None,
            "매도일": row['매도일'].strftime('%Y-%m-%d') if not pd.isna(row['매도일']) else None,
        })

    result_df = get_stock_data_summary(stocks)
    print(result_df)

    end_time = datetime.now(seoul_tz)
    print(f"종료 시간: {end_time}")
    print(f"총 소요 시간: {end_time - start_time}")

    result_df.to_excel("stock_summary_final.xlsx")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Processing 111: 대봉엘에스_111
Processing 112: 에이치브이엠_112
Processing 838: 펨트론_838
Processing 113: 아이엠비디엑스_113
Processing 839: 알테오젠_839
Processing 114: 지아이이노베이션_114
Processing 840: 한올바이오파마_840
Processing 115: 드래곤플라이_115
Processing 841: 메디톡스_841
Processing 842: 나이벡_842
Processing 116: 엑스게이트_116
Processing 843: 동신건설_843
Processing 117: 제닉_117
Processing 844: 제이앤티씨_844
Processing 118: 제이투케이바이오_118
Processing 119: 지역난방공사_119Processing 845: 샌즈랩_845

Processing 120: 한싹_120
Processing 846: 메지온_846
Processing 121: 애드바이오텍_121
Processing 847: 세방전지_847
Processing 122: 이브이첨단소재_122
Processing 848: 오성첨단소재_848
Processing 849: SAMG엔터_849
Processing 123: 덱스터_123
Processing 850: 쿠쿠홀딩스_850
Processing 124: 예스24_124
Processing 125: 에스비비테크_125
Processing 126: 에스티오_126
Processing 127: 레인보우로보틱스_127
Processing 851: 현대오토에버_851
Processing 128: 아티스트유나이티드_128
Processing 129: 엔피디_129
Processing 852: 알에스오토메이션_852
Processing 130: 아이티아이즈_130
Processing 853: 유진테크_853
Processing 131: 성도이엔지_

TypeError: strptime() argument 1 must be str, not None